<a href="https://colab.research.google.com/github/RyutoYoda/WOOD_LIFE/blob/main/smbc_woood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/SMBC')

In [2]:

!pip install lightgbm \
    --no-binary lightgbm \
    --no-cache lightgbm \
    --config-settings=cmake.define.USE_CUDA=ON

In [65]:
import numpy as np

In [7]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.impute import SimpleImputer

In [46]:
# データの読み込み
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [12]:
train.head()

,Unnamed: 0,created_at,tree_dbh,curb_loc,health,steward,guards,sidewalk,user_type,problems,...,curb_loc_encoded,steward_encoded,guards_encoded,sidewalk_encoded,user_type_encoded,problems_encoded,nta_encoded,nta_name_encoded,boroname_encoded,zip_city_encoded
0,0,2015-06-29,14,OnCurb,1,Unknown,Unknown,Damage,Volunteer,Unknown,...,0.858391,0.864207,0.870307,0.852717,0.849674,0.870130,0.905405,0.905405,0.858186,0.905405
1,1,2016-09-21,5,OnCurb,1,3or4,Helpful,NoDamage,Volunteer,Unknown,...,0.858391,0.857275,0.824110,0.862663,0.849674,0.870130,0.864865,0.864865,0.854626,0.854626
2,2,2015-09-13,26,OnCurb,2,Unknown,Unknown,NoDamage,Volunteer,StonesBranchLights,...,0.858391,0.864207,0.870307,0.862663,0.849674,0.809711,0.836667,0.836667,0.861044,0.861044
3,3,2016-05-09,15,OnCurb,0,Unknown,Unknown,Damage,NYC Parks Staff,Unknown,...,0.858391,0.864207,0.870307,0.852717,0.863000,0.870130,0.888579,0.888579,0.861044,0.861044
4,4,2016-06-24,23,OnCurb,1,Unknown,Unknown,NoDamage,Volunteer,Stones,...,0.858391,0.864207,0.870307,0.862663,0.849674,0.826048,0.784810,0.784810,0.860577,0.858844


In [57]:
train.isnull().sum()

Unnamed: 0           0
created_at           0
tree_dbh             0
curb_loc             0
health               0
steward              0
guards               0
sidewalk             0
user_type            0
problems             0
spc_common           0
spc_latin            0
nta                  0
nta_name             0
borocode             0
boro_ct              0
boroname             0
zip_city             0
cb_num               0
st_senate            0
st_assem             0
cncldist             0
curb_loc_encoded     0
steward_encoded      0
guards_encoded       0
sidewalk_encoded     0
user_type_encoded    0
problems_encoded     0
nta_encoded          0
nta_name_encoded     0
boroname_encoded     0
zip_city_encoded     0
dtype: int64

In [ ]:
test.isnull().sum()

In [42]:
test.head()

,Unnamed: 0,created_at,tree_dbh,curb_loc,steward,guards,sidewalk,user_type,problems,spc_common,...,curb_loc_encoded,steward_encoded,guards_encoded,sidewalk_encoded,user_type_encoded,problems_encoded,nta_encoded,nta_name_encoded,boroname_encoded,zip_city_encoded
0,19984,2015-09-08,15,OnCurb,Unknown,Unknown,Damage,NYC Parks Staff,Unknown,Callery pear,...,0.858716,0.868645,0.875605,0.856505,0.858257,0.869517,0.828571,0.828571,0.853019,0.853019
1,19985,2015-10-15,5,OnCurb,1or2,Unknown,NoDamage,Volunteer,Unknown,cherry,...,0.858716,0.824371,0.875605,0.859598,0.845709,0.869517,0.807692,0.807692,0.862099,0.862099
2,19986,2016-08-03,4,OnCurb,Unknown,Unsure,NoDamage,Volunteer,Unknown,littleleaf linden,...,0.858716,0.868645,0.747989,0.859598,0.845709,0.869517,0.816327,0.816327,0.858213,0.818182
3,19987,2015-08-06,7,OnCurb,Unknown,Unknown,NoDamage,TreesCount Staff,Unknown,dawn redwood,...,0.858716,0.868645,0.875605,0.859598,0.881998,0.869517,0.916667,0.916667,0.853019,0.853019
4,19988,2015-10-21,6,OnCurb,1or2,Helpful,NoDamage,TreesCount Staff,Stones,purple-leaf plum,...,0.858716,0.824371,0.811271,0.859598,0.881998,0.829204,0.890909,0.890909,0.853019,0.853019


In [47]:
# 欠損値の処理
imputer = SimpleImputer(strategy="constant", fill_value="Unknown")
train[['steward', 'guards', 'problems']] = imputer.fit_transform(train[['steward', 'guards', 'problems']])
test[['steward', 'guards', 'problems']] = imputer.transform(test[['steward', 'guards', 'problems']])

In [48]:
# ターゲットエンコーディングを実行するカテゴリ変数のリスト
categorical_columns = ['curb_loc', 'steward','guards', 'sidewalk','user_type','problems','nta', 'nta_name','boroname','zip_city']  # カテゴリ変数の列名を適切なものに変更

# カテゴリごとにターゲットエンコーディングを実行
for col in categorical_columns:
    target_mapping = train.groupby(col)['health'].mean().to_dict()
    train[col + '_encoded'] = train[col].map(target_mapping)


In [49]:
# カテゴリ変数ごとにトレーニングデータからのエンコーディング情報を計算
encoding_mapping = {}
for col in categorical_columns:
    encoding_mapping[col] = train.groupby(col)['health'].mean().to_dict()

In [50]:
# テストデータに対して新たにターゲットエンコーディングを実行
for col in categorical_columns:
    test[col + '_encoded'] = test[col].map(encoding_mapping[col])

In [51]:
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train, test_size=0.2, stratify=train['health'], random_state=82)

In [58]:
# 使用する特徴量の選択
select_cols = ['tree_dbh', 'curb_loc_encoded','steward_encoded','guards_encoded','sidewalk_encoded','user_type_encoded','problems_encoded','nta_encoded','nta_name_encoded','boroname_encoded','zip_city_encoded']

# 目的変数とそれ以外に学習用データを分割
x_train = train[select_cols]
y_train = train['health']
x_valid = valid[select_cols]
y_valid = valid['health']

##ランダムフォレストモデル

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

model = RandomForestClassifier()
model.fit(x_train, y_train)

valid_predictions = model.predict(x_valid)
valid_f1 = f1_score(y_valid, valid_predictions, average='macro')
print(f"Validation F1 Score (Macro): {valid_f1}")

Validation F1 Score (Macro): 0.32337385747172304


In [60]:
# テストデータから不要な特徴量を削除
test = test[select_cols]

In [61]:
pred = model.predict(test)
print(pred[:5])

[1 1 1 1 0]


In [70]:
sample_submit = pd.read_csv('sample_submission.csv', index_col=0, header=None) # 応募用サンプルファイル

In [71]:
sample_submit[1] = pred
sample_submit.to_csv('/content/drive/MyDrive/SMBC/submission.csv', header=None)